## Funzioni di servizio

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import time
import random
import math
import copy

In [ ]:
def create_graph(num_nodes):

    # crea grafo completo: gli archi rappresentano già le distanze minime tra i nodi
    G = nx.complete_graph(num_nodes)
    max_coord = num_nodes*5
    range_pos = (0, max_coord)

    # attributi nodi
    for node in G.nodes:
        # posizione random
        pos = (random.uniform(*range_pos), random.uniform(*range_pos))
        G.nodes[node]['pos'] = pos
        G.nodes[node]['station'] = False
        G.nodes[node]['depot'] = False

    # deposito circa al centro del grafo
    depot_node = min(G.nodes, key=lambda n: math.dist(G.nodes[n]['pos'], (max_coord/2, max_coord/2)))
    G.nodes[depot_node]['depot'] = True

    # stazioni random escludendo il deposito, 20% dei nodi
    num_stations = num_nodes // 5
    candidates = [n for n in G.nodes if n != depot_node]
    stations = random.sample(candidates, num_stations)
    for node in stations:
        G.nodes[node]['station'] = True

    # peso archi + distanza euclidea tra i nodi
    for u, v in G.edges:
        pos_u = G.nodes[u]['pos']
        pos_v = G.nodes[v]['pos']
        dist = math.dist(pos_u, pos_v)
        G.edges[u, v]['weight'] = dist

    return G

def route_time(G, route):

    total = 0
    refillTime = 0
    used_fuel = 0
    full_route = route + [route[0]]
    for u, v in zip(full_route, full_route[1:]):
        if u == v: # non c'è l'arco del nodo a se stesso
            continue

        used_fuel += G[u][v]['weight']
        total += G[u][v]['weight']

        if G.nodes[v].get('station'):
            refillTime += used_fuel
            used_fuel = 0

    return total + refillTime

def plot_tsp(G, route, routetime, title, calctime):

    pos = nx.get_node_attributes(G, 'pos')

    color_map = []
    for node in G.nodes:
        if G.nodes[node]['depot']:
            color_map.append('yellow')
        elif G.nodes[node]['station']:
            color_map.append('green')
        else:
            color_map.append('blue')

    plt.figure(figsize=(10, 8))
    nx.draw_networkx_nodes(G, pos, node_color=color_map, node_size=200)

    nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.3)
    nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')
    best_edges = [(route[i], route[i + 1]) for i in range(len(route) - 1)]
    best_edges.append((route[-1], route[0]))
    nx.draw_networkx_edges(G, pos, edgelist=best_edges, edge_color='red', width=2)
    plt.title(f"{title}\nRoute time: {routetime:.2f}, Calc time: {calctime:.6f}s", fontsize=14)
    plt.axis('off')
    plt.show()

def fuel_check(G, route, fuel):

    full = fuel

    for u, v in zip(route, route[1:]):

        if u == v:
            continue

        dist = G[u][v]['weight']

        if dist > fuel:
            return False

        fuel -= dist

        if G.nodes[v].get('station'):
            fuel = full

    return True

## Euristiche Nearest Neighbor e Two-opt

In [ ]:
def solve_tsp_nearest_neighbor(G, fuel):

    full = fuel
    refillTime = 0
    depot_node = next(node for node, data in G.nodes(data=True) if data.get('depot', False))
    #print(f"Depot: {depot_node}")
    stations = [n for n in G.nodes if G.nodes[n].get('station', False)]
    #print(f"Stations: {stations}")

    route = [depot_node]
    unvisited = set(G.nodes) - {depot_node} - set(stations)

    fuel = full
    visits = len(unvisited)+1
    while visits > 0:
        current_city = route[-1]
        if visits == 1:
            next_city = depot_node
            visits -= 1
        else: next_city = min(unvisited, key=lambda x: G[current_city][x]['weight'])
        #print(f"Current city: {current_city}, Next city: {next_city}")
        # cerca la stazione più vicina
        current_station = min(stations, key=lambda x: G[current_city][x]['weight'])
        next_station = min(stations, key=lambda x: G[next_city][x]['weight'])
        # se la distanza da current_city a next_city più quella tra next_city e la stazione è minore del carburante fai carburante nella stazione più vicina a current_city
        # calcola distanza tra current_city e next_city
        dist_currC_nextC = G[current_city][next_city]['weight']
        # calcola distanza tra next_city e stazione_next
        dist_nextC_nextS = G[next_city][next_station]['weight']
        if ((dist_currC_nextC + dist_nextC_nextS > fuel) and next_city != depot_node):
            # bisogna fare carburante
            route.append(current_station)
            # sottrai da carburante il peso dell'arco
            fuel -= G[current_city][current_station]['weight']
            refillTime += full - fuel
            fuel = full
            route.append(next_city)
            fuel -= G[current_station][next_city]['weight']
            unvisited.remove(next_city)
            visits -= 1


        elif ((dist_currC_nextC > fuel) and next_city == depot_node):
            # bisogna fare carburante
            route.append(current_station)
            # sottrai da carburante il peso dell'arco
            fuel -= G[current_city][current_station]['weight']
            # aggiungi carburante
            refillTime += full - fuel
            fuel = full
            route.append(depot_node)
            fuel -= G[current_station][depot_node]['weight']
            visits -= 1

        else:
            # non è necessario fare carburante
            # sottrai da carburante il peso dell'arco
            fuel -= G[current_city][next_city]['weight']
            route.append(next_city)
            if (next_city != depot_node): unvisited.remove(next_city)
            visits -= 1

    routeTime = route_time(G, route)

    return route, routeTime

def two_opt(G, initial_route, initial_route_time, fuel):

    best_route = initial_route
    best_time = initial_route_time

    improved = True
    while improved:
        improved = False
        N = len(best_route)

        for i in range(1, N - 2):
            for j in range(i + 1, N - 1):
                new_route = (
                    best_route[:i] +
                    best_route[i:j+1][::-1] +
                    best_route[j+1:]
                )

                new_time = route_time(G, new_route)
                if new_time < best_time and fuel_check(G, new_route, fuel):
                    best_route, best_time = new_route, new_time
                    improved = True
                    break
            if improved:
                break
    return best_route, best_time

## Euristica Simulated Annealing

In [ ]:
def energy_check(G, tour, K, charging_stations):
    energy = K
    for i in range(len(tour) - 1):
        u, v = tour[i], tour[i + 1]

        if u == v:
            continue

        energy -= G.edges[u, v]['weight']
        if energy < 0:
            return False
        if v in charging_stations:
            energy = K
    return True

def swap(G, tour, K, charging_stations, max_attempts=300):
    n = len(tour)
    if n <= 3:
        return tour

    attempts = 0
    while attempts < max_attempts:
        new_tour = copy.deepcopy(tour)
        i, j = sorted(random.sample(range(1, n - 1), 2))
        new_tour[i], new_tour[j] = new_tour[j], new_tour[i]

        if energy_check(G, new_tour, K, charging_stations):
            routeTime = route_time(G, new_tour)
            return new_tour, routeTime

        attempts += 1

    return tour, route_time(G, tour)

def simulated_annealing(G, carburante, route, route_time, charging_stations,
                        T_init=138.27750786775712, T_min=0.5584219747554267,
                        alpha=0.9051152229224724, max_no_improve=1941):
    current_solution = route
    best_solution = current_solution
    current_cost = route_time
    best_cost = current_cost
    T = T_init
    no_improve = 0

    while T > T_min and no_improve < max_no_improve:
        swap_solution, swap_cost = swap(G, current_solution, carburante, charging_stations)

        if fuel_check(G, swap_solution, carburante):
            new_solution, new_cost = two_opt(G, swap_solution, swap_cost, carburante)
            delta = new_cost - current_cost

            if delta < 0 or random.random() < math.exp(-delta / T):
                current_solution = new_solution
                current_cost = new_cost

                if new_cost < best_cost:
                    best_solution = new_solution
                    best_cost = new_cost
                    no_improve = 0
                else:
                    no_improve += 1
            else:
                no_improve += 1
        else:
            continue
        T *= alpha

    return best_solution, best_cost

## Esecuzione TSP

In [ ]:
random.seed(42)

num_nodes = 200
G = create_graph(num_nodes)
depot_node = next(node for node, data in G.nodes(data=True) if data.get('depot', False))

start_nn = time.time()
nn_route, nn_routetime = solve_tsp_nearest_neighbor(G, fuel=1000)
end_nn = time.time() - start_nn

start_time_2_opt = time.time()
two_opt_route, two_opt_route_time = two_opt(G, nn_route, nn_routetime, fuel=1000)
end_time_2_opt = time.time() - start_time_2_opt

start_sa = time.time()
charging_stations = [node for node, data in G.nodes(data=True) if data.get('stazione', False)]
sa_route, sa_time = simulated_annealing(G, 1000, nn_route, nn_routetime, charging_stations)
end_sa = time.time() - start_sa

print(f"Number of nodes: {num_nodes}")
print(f"Nearest Neighbor\nRoute: {nn_route}\nRoute time: {nn_routetime:.2f}\nCalc time: {end_nn:.6f}s")
print(f"NN + 2-opt\nRoute: {two_opt_route}\nRoute time: {two_opt_route_time:.2f}\nCalc time: {end_time_2_opt:.6f}s")
print(f"Simulated Annealing\nRoute: {sa_route}\nRoute time: {sa_time:.2f}\nCalc time: {end_sa:.6f}s")

In [ ]:
def plot_tsp_ax(G, route, routetime, title, calctime, ax):
    pos = nx.get_node_attributes(G, 'pos')
    # colori nodi
    color_map = []
    for node in G.nodes:
        if G.nodes[node]['deposito']:
            color_map.append('yellow')
        elif G.nodes[node]['stazione']:
            color_map.append('green')
        else:
            color_map.append('blue')
    # disegno
    nx.draw_networkx_nodes(G, pos, node_color=color_map, node_size=100, ax=ax)
    nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.3, ax=ax)
    best_edges = [(route[i], route[i+1]) for i in range(len(route)-1)] + [(route[-1], route[0])]
    nx.draw_networkx_edges(G, pos, edgelist=best_edges, edge_color='red', width=2, ax=ax)
    ax.set_title(f"{title}\nTsp time: {routetime:.2f}s\nCalc: {calctime:.4f}s", fontsize=10)

# --- 1) Plotto affiancato per n nodi ---
def plot_comparison(n):
    num_nodes = n
    G = create_graph(num_nodes)
    depot = next(n for n,d in G.nodes(data=True) if d.get('deposito'))

    # NN
    t0 = time.time()
    nn_route, nn_rt = solve_tsp_nearest_neighbor(G, fuel=1000)
    t_nn = time.time() - t0
    # Two-opt
    t0 = time.time()
    to_route, to_rt = two_opt(G, nn_route, nn_rt, fuel=1000)
    t_to = time.time() - t0
    # Simulated Annealing
    t0 = time.time()
    charging_stations = [node for node, data in G.nodes(data=True) if data.get('stazione', False)]
    sa_route, sa_rt = simulated_annealing(G, 1000, to_route, to_rt, charging_stations)
    t_sa = time.time() - t0

    fig, axes = plt.subplots(1, 3, figsize=(18,6))
    plot_tsp_ax(G, nn_route, nn_rt, "Nearest Neighbor", t_nn, axes[0])
    plot_tsp_ax(G, to_route, to_rt, "NN + 2-Opt", t_to, axes[1])
    plot_tsp_ax(G, sa_route, sa_rt, "Simulated Annealing", t_sa, axes[2])
    plt.tight_layout()
    plt.show()

# --- 2) Benchmark su nodi 10–200 ---
def benchmark_algorithms(node_values):
    times_calc = {'NN':[], '2OPT':[], 'SA':[]}
    times_route = {'NN':[], '2OPT':[], 'SA':[]}

    for n in node_values:
        G = create_graph(n)
        depot = next(n0 for n0,d in G.nodes(data=True) if d.get('deposito'))

        # NN
        t0 = time.time()
        route_nn, rt_nn = solve_tsp_nearest_neighbor(G, fuel=1000)
        t_nn = time.time() - t0

        # 2OPT
        t0 = time.time()
        route_to, rt_to = two_opt(G, route_nn, rt_nn, fuel=1000)
        t_to = time.time() - t0

        # SA
        t0 = time.time()
        charging_stations = [node for node, data in G.nodes(data=True) if data.get('stazione', False)]
        sa_route, sa_rt = simulated_annealing(G, 1000, route_to, rt_to, charging_stations)
        t_sa = time.time() - t0

        times_calc['NN'].append(t_nn)
        times_calc['2OPT'].append(t_to)
        times_calc['SA'].append(t_sa)
        times_route['NN'].append(rt_nn)
        times_route['2OPT'].append(rt_to)
        times_route['SA'].append(sa_rt)

    return times_calc, times_route

def plot_benchmarks(node_values, times_calc, times_route):
    # 2.a Tempo di calcolo
    plt.figure(figsize=(8,5))
    plt.plot(node_values, times_calc['NN'], label='Nearest Neighbor')
    plt.plot(node_values, times_calc['2OPT'], label='NN + 2-Opt')
    plt.plot(node_values, times_calc['SA'], label='Simulated Annealing')
    plt.xlabel('Numero di nodi')
    plt.ylabel('Tempo di calcolo (s)')
    plt.title('Benchmark: Tempo di calcolo vs Nodi')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # 2.b Tempo di rotta
    plt.figure(figsize=(8,5))
    plt.plot(node_values, times_route['NN'], label='Nearest Neighbor')
    plt.plot(node_values, times_route['2OPT'], label='NN + 2-Opt')
    plt.plot(node_values, times_route['SA'], label='Simulated Annealing')
    plt.xlabel('Numero di nodi')
    plt.ylabel('Route time (s)')
    plt.title('Benchmark: Route Time vs Nodi')
    plt.legend()

    plt.tight_layout()
    plt.show()


plot_comparison(50)

nodes = list(range(10, 51, 10))
calc_times, route_times = benchmark_algorithms(nodes)
plot_benchmarks(nodes, calc_times, route_times)